In [208]:
import hashlib

def md5_min_js(str_passwd, str_ver):
    def md5_rotate_left(l_value, i_shift_bits):
        return (l_value << i_shift_bits) | (l_value >> (32 - i_shift_bits))

    def md5_add_unsigned(l_x, l_y):
        l_x4 = l_x & 0x40000000
        l_y4 = l_y & 0x40000000
        l_x8 = l_x & 0x80000000
        l_y8 = l_y & 0x80000000
        l_result = (l_x & 0x3FFFFFFF) + (l_y & 0x3FFFFFFF)
        if l_x4 & l_y4:
            return l_result ^ 0x80000000 ^ l_x8 ^ l_y8
        if l_x4 | l_y4:
            if l_result & 0x40000000:
                return l_result ^ 0xC0000000 ^ l_x8 ^ l_y8
            else:
                return l_result ^ 0x40000000 ^ l_x8 ^ l_y8
        else:
            return l_result ^ l_x8 ^ l_y8

    def md5_f(x, y, z):
        return (x & y) | (~x & z)

    def md5_g(x, y, z):
        return (x & z) | (y & ~z)

    def md5_h(x, y, z):
        return x ^ y ^ z

    def md5_i(x, y, z):
        return y ^ (x | ~z)

    def md5_ff(a, b, c, d, x, s, ac):
        a = md5_add_unsigned(a, md5_add_unsigned(md5_add_unsigned(md5_f(b, c, d), x), ac))
        return md5_add_unsigned(md5_rotate_left(a, s), b)

    def md5_gg(a, b, c, d, x, s, ac):
        a = md5_add_unsigned(a, md5_add_unsigned(md5_add_unsigned(md5_g(b, c, d), x), ac))
        return md5_add_unsigned(md5_rotate_left(a, s), b)

    def md5_hh(a, b, c, d, x, s, ac):
        a = md5_add_unsigned(a, md5_add_unsigned(md5_add_unsigned(md5_h(b, c, d), x), ac))
        return md5_add_unsigned(md5_rotate_left(a, s), b)

    def md5_ii(a, b, c, d, x, s, ac):
        a = md5_add_unsigned(a, md5_add_unsigned(md5_add_unsigned(md5_i(b, c, d), x), ac))
        return md5_add_unsigned(md5_rotate_left(a, s), b)

    def md5_convert_to_word_array(string):
        l_message_length = len(string)
        l_number_of_words_temp1 = l_message_length + 8
        l_number_of_words_temp2 = (l_number_of_words_temp1 - (l_number_of_words_temp1 % 64)) // 64
        l_number_of_words = (l_number_of_words_temp2 + 1) * 16
        l_word_array = [0] * l_number_of_words
        l_byte_position = 0
        l_byte_count = 0

        while l_byte_count < l_message_length:
            l_word_count = l_byte_count // 4
            l_byte_position = (l_byte_count % 4) * 8
            l_word_array[l_word_count] |= ord(string[l_byte_count]) << l_byte_position
            l_byte_count += 1

        l_word_count = l_byte_count // 4
        l_byte_position = (l_byte_count % 4) * 8
        l_word_array[l_word_count] |= 0x80 << l_byte_position
        l_word_array[l_number_of_words - 2] = l_message_length << 3
        l_word_array[l_number_of_words - 1] = l_message_length >> 29

        return l_word_array

    def md5_word_to_hex(l_value):
        word_to_hex_value = ""
        for l_count in range(4):
            l_byte = (l_value >> (l_count * 8)) & 255
            word_to_hex_value += f"{l_byte:02x}"
        return word_to_hex_value

    def md5_utf8_encode(string):
        return string.encode('utf-8')

    def hex_md5(string, ver):
        S11, S12, S13, S14 = 7, 12, 17, 22
        S21, S22, S23, S24 = 5, 9, 14, 20
        S31, S32, S33, S34 = 4, 11, 16, 23
        S41, S42, S43, S44 = 6, 10, 15, 21

        string = md5_utf8_encode(string + ("" if ver == "1.8" else "{Urp602019}")).decode('utf-8')
        x = md5_convert_to_word_array(string)
        a, b, c, d = 0x67452301, 0xEFCDAB89, 0x98BADCFE, 0x10325476

        for k in range(0, len(x), 16):
            AA, BB, CC, DD = a, b, c, d

            a = md5_ff(a, b, c, d, x[k + 0], S11, 0xD76AA478)
            d = md5_ff(d, a, b, c, x[k + 1], S12, 0xE8C7B756)
            c = md5_ff(c, d, a, b, x[k + 2], S13, 0x242070DB)
            b = md5_ff(b, c, d, a, x[k + 3], S14, 0xC1BDCEEE)
            a = md5_ff(a, b, c, d, x[k + 4], S11, 0xF57C0FAF)
            d = md5_ff(d, a, b, c, x[k + 5], S12, 0x4787C62A)
            c = md5_ff(c, d, a, b, x[k + 6], S13, 0xA8304613)
            b = md5_ff(b, c, d, a, x[k + 7], S14, 0xFD469501)
            a = md5_ff(a, b, c, d, x[k + 8], S11, 0x698098D8)
            d = md5_ff(d, a, b, c, x[k + 9], S12, 0x8B44F7AF)
            c = md5_ff(c, d, a, b, x[k + 10], S13, 0xFFFF5BB1)
            b = md5_ff(b, c, d, a, x[k + 11], S14, 0x895CD7BE)
            a = md5_ff(a, b, c, d, x[k + 12], S11, 0x6B901122)
            d = md5_ff(d, a, b, c, x[k + 13], S12, 0xFD987193)
            c = md5_ff(c, d, a, b, x[k + 14], S13, 0xA679438E)
            b = md5_ff(b, c, d, a, x[k + 15], S14, 0x49B40821)

            a = md5_gg(a, b, c, d, x[k + 1], S21, 0xF61E2562)
            d = md5_gg(d, a, b, c, x[k + 6], S22, 0xC040B340)
            c = md5_gg(c, d, a, b, x[k + 11], S23, 0x265E5A51)
            b = md5_gg(b, c, d, a, x[k + 0], S24, 0xE9B6C7AA)
            a = md5_gg(a, b, c, d, x[k + 5], S21, 0xD62F105D)
            d = md5_gg(d, a, b, c, x[k + 10], S22, 0x2441453)
            c = md5_gg(c, d, a, b, x[k + 15], S23, 0xD8A1E681)
            b = md5_gg(b, c, d, a, x[k + 4], S24, 0xE7D3FBC8)
            a = md5_gg(a, b, c, d, x[k + 9], S21, 0x21E1CDE6)
            d = md5_gg(d, a, b, c, x[k + 14], S22, 0xC33707D6)
            c = md5_gg(c, d, a, b, x[k + 3], S23, 0xF4D50D87)
            b = md5_gg(b, c, d, a, x[k + 8], S24, 0x455A14ED)
            a = md5_gg(a, b, c, d, x[k + 13], S21, 0xA9E3E905)
            d = md5_gg(d, a, b, c, x[k + 2], S22, 0xFCEFA3F8)
            c = md5_gg(c, d, a, b, x[k + 7], S23, 0x676F02D9)
            b = md5_gg(b, c, d, a, x[k + 12], S24, 0x8D2A4C8A)

            a = md5_hh(a, b, c, d, x[k + 5], S31, 0xFFFA3942)
            d = md5_hh(d, a, b, c, x[k + 8], S32, 0x8771F681)
            c = md5_hh(c, d, a, b, x[k + 11], S33, 0x6D9D6122)
            b = md5_hh(b, c, d, a, x[k + 14], S34, 0xFDE5380C)
            a = md5_hh(a, b, c, d, x[k + 1], S31, 0xA4BEEA44)
            d = md5_hh(d, a, b, c, x[k + 4], S32, 0x4BDECFA9)
            c = md5_hh(c, d, a, b, x[k + 7], S33, 0xF6BB4B60)
            b = md5_hh(b, c, d, a, x[k + 10], S34, 0xBEBFBC70)
            a = md5_hh(a, b, c, d, x[k + 13], S31, 0x289B7EC6)
            d = md5_hh(d, a, b, c, x[k + 0], S32, 0xEAA127FA)
            c = md5_hh(c, d, a, b, x[k + 3], S33, 0xD4EF3085)
            b = md5_hh(b, c, d, a, x[k + 6], S34, 0x4881D05)
            a = md5_hh(a, b, c, d, x[k + 9], S31, 0xD9D4D039)
            d = md5_hh(d, a, b, c, x[k + 12], S32, 0xE6DB99E5)
            c = md5_hh(c, d, a, b, x[k + 15], S33, 0x1FA27CF8)
            b = md5_hh(b, c, d, a, x[k + 2], S34, 0xC4AC5665)

            a = md5_ii(a, b, c, d, x[k + 0], S41, 0xF4292244)
            d = md5_ii(d, a, b, c, x[k + 7], S42, 0x432AFF97)
            c = md5_ii(c, d, a, b, x[k + 14], S43, 0xAB9423A7)
            b = md5_ii(b, c, d, a, x[k + 5], S44, 0xFC93A039)
            a = md5_ii(a, b, c, d, x[k + 12], S41, 0x655B59C3)
            d = md5_ii(d, a, b, c, x[k + 3], S42, 0x8F0CCC92)
            c = md5_ii(c, d, a, b, x[k + 10], S43, 0xFFEFF47D)
            b = md5_ii(b, c, d, a, x[k + 1], S44, 0x85845DD1)
            a = md5_ii(a, b, c, d, x[k + 8], S41, 0x6FA87E4F)
            d = md5_ii(d, a, b, c, x[k + 15], S42, 0xFE2CE6E0)
            c = md5_ii(c, d, a, b, x[k + 6], S43, 0xA3014314)
            b = md5_ii(b, c, d, a, x[k + 13], S44, 0x4E0811A1)
            a = md5_ii(a, b, c, d, x[k + 4], S41, 0xF7537E82)
            d = md5_ii(d, a, b, c, x[k + 11], S42, 0xBD3AF235)
            c = md5_ii(c, d, a, b, x[k + 2], S43, 0x2AD7D2BB)
            b = md5_ii(b, c, d, a, x[k + 9], S44, 0xEB86D391)

            a = md5_add_unsigned(a, AA)
            b = md5_add_unsigned(b, BB)
            c = md5_add_unsigned(c, CC)
            d = md5_add_unsigned(d, DD)

        return (md5_word_to_hex(a) + md5_word_to_hex(b) + md5_word_to_hex(c) + md5_word_to_hex(d)).lower()
    # 如果 盐 = 1.8 那么正常处理，如果 盐 != 1.8 那么加上 {Urp602019}

    return hex_md5(str_passwd, str_ver)
password = '0'

In [209]:
print(password)

0


In [210]:
print(md5_min_js(password, ''))

c5d5c5f1fe3ec7f173578fcc50fb92ea


In [211]:
print(md5_min_js(password, '1.8'))

cfcd208495d565ef66e7dff9f98764da


In [212]:
print(hashlib.md5(password.encode("utf-8")).hexdigest())

cfcd208495d565ef66e7dff9f98764da


In [213]:
print(hashlib.md5((password+"{Urp602019}").encode("utf-8")).hexdigest())

c5d5c5f1fe3ec7f173578fcc50fb92ea
